In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [11]:
year = 2018
month = 1
df_append_start = pd.DataFrame()
df_append_end = pd.DataFrame()

while year < 2020 and month < 14:
    # generate path & read CSV
    path = f'csv/{str(year) + str(month).zfill(2)}-citibike-tripdata.csv'
    df = pd.read_csv(path)
    
    # filter out rides longer than 1 hr
    df['tripduration'] = df['tripduration'].astype(int)
    df = df[(df.tripduration <= 3600)] 
    
    # convert start time to datetime to extract year & month
    df['starttime'] = pd.to_datetime(df['starttime'])
    df['trip_year'] = df['starttime'].dt.strftime('%Y').astype(int)
    df['trip_month'] = df['starttime'].dt.strftime('%m').astype(int)
    
    # convert trip duration to minutes
    #df['tripduration_mins'] = df['tripduration'].apply(lambda x: x / 60)
    
    # group by month, count rides
    df_reduced = df[['start station name','start station latitude','start station longitude', 'end station name', 'end station latitude', 'end station longitude', 'trip_month', 'trip_year', 'usertype']]
    df_reduced_start = df_reduced.groupby(['trip_year', 'start station name','start station latitude','start station longitude']).count().reset_index()
    
    df_reduced_end = df_reduced.groupby(['trip_year', 'end station name', 'end station latitude', 'end station longitude']).count().reset_index()
    
    df_append_start = df_append_start.append(df_reduced_start)
    df_append_end = df_append_end.append(df_reduced_end)
    
    print(f' Month: {month}, Shape: {df.shape}, Append Shape: {df_append_start.shape}')
    month += 1
    

    if (month == 13):
        month = 1
        year += 1

 Month: 1, Shape: (716571, 17), Append Shape: (763, 9)
 Month: 2, Shape: (839734, 17), Append Shape: (1525, 9)
 Month: 3, Shape: (971645, 17), Append Shape: (2292, 9)
 Month: 4, Shape: (1294222, 17), Append Shape: (3057, 9)
 Month: 5, Shape: (1803727, 17), Append Shape: (3824, 9)
 Month: 6, Shape: (1929984, 17), Append Shape: (4589, 9)
 Month: 7, Shape: (1888771, 17), Append Shape: (5356, 9)
 Month: 8, Shape: (1954519, 17), Append Shape: (6126, 9)
 Month: 9, Shape: (1857267, 17), Append Shape: (6894, 9)
 Month: 10, Shape: (1863633, 17), Append Shape: (7659, 9)
 Month: 11, Shape: (1251410, 17), Append Shape: (8420, 9)
 Month: 12, Shape: (1010543, 17), Append Shape: (9182, 9)
 Month: 1, Shape: (962971, 17), Append Shape: (9949, 9)
 Month: 2, Shape: (939109, 17), Append Shape: (10716, 9)
 Month: 3, Shape: (1318379, 17), Append Shape: (11485, 9)
 Month: 4, Shape: (1746403, 17), Append Shape: (12269, 9)
 Month: 5, Shape: (1903280, 17), Append Shape: (13062, 9)
 Month: 6, Shape: (2099976, 17

In [16]:
list(df_append_start)

['trip_year',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station name',
 'end station latitude',
 'end station longitude',
 'trip_month',
 'usertype']

In [17]:
df_start = df_append_start.copy()
df_end = df_append_end.copy()

df_start.rename(columns={'usertype': 'ride_count'}, inplace=True)
df_start.drop(columns=['end station name', 'end station latitude', 'end station longitude', 'trip_month'], inplace=True)
df_end.rename(columns={'usertype': 'ride_count'}, inplace=True)
df_end.drop(columns=['start station name','start station latitude','start station longitude', 'trip_month'], inplace=True)

In [19]:
df_start.head()

,trip_year,start station name,start station latitude,start station longitude,ride_count
0,2018,1 Ave & E 110 St,40.792327,-73.938300,310
1,2018,1 Ave & E 16 St,40.732219,-73.981656,3144
2,2018,1 Ave & E 18 St,40.733812,-73.980544,2202
3,2018,1 Ave & E 30 St,40.741444,-73.975361,2356
4,2018,1 Ave & E 44 St,40.750020,-73.969053,1415


In [20]:
df_start.to_csv('csv/start_station_counts.csv')
df_end.to_csv('csv/end_station_counts.csv')